In [1]:
# Hello Inkvestigators!  This is our Jupyter Notebook 

In [5]:
#running a simple multilayer perceptron neural network
import pandas as pd





In [40]:
df = pd.read_csv('train.csv')
X_test = pd.read_csv('test.csv')

In [32]:
# Assigning which data points are x and which are y
# x is all rows(every digit in the set) and all columns except the first
# y is the prediction which is listed in the first column "0"
#float32 keeps pytorch from treating values as integers
#long converts the predictions to integers
import torch

X = torch.tensor(df.values[:, 1:], dtype=torch.float32) / 255.0

Y = torch.tensor(df.values [:, 0], dtype=torch.long)

In [35]:
# Assigning the shape of the neural network model
import torch.nn as nn
model = nn.Sequential(
    nn.Linear(784, 784),
    nn.ReLU(),
    nn.Linear(784, 10),
    # pytorch automatically uses a softmax activation function here where all probabilities of the ten nodes add up to 1.0
)

In [36]:
# epoch is number of times we run the model over all the digits
epochs = 10
# batch size is number of digits run through the model at a time
batch_size = 100
#batches per epoch is how many different batches of digits are sent through the model to complete one epoch 
batches_per_epoch = len(X) // batch_size

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(epochs):
    for i in range(batches_per_epoch):

        #get a batch
        start = i * batch_size
        X_batch = X[start:start+batch_size]
        Y_batch = Y[start:start+batch_size]

        #forward pass.  we compare the model's prediction against the actual y
        Y_pred = model(X_batch)
        loss = loss_fn(Y_pred, Y_batch)

        #backward pass.  zero gradiant is the where the derivative of the graph for y is at its lowest point
        optimizer.zero_grad()
        loss.backward()

        #update weights
        optimizer.step()

    # Validation.  argmax is trying to find the node (1-9) that has a prediction probability closest to 1 
    model.eval()
    y_pred = model (X)
    acc = (torch.argmax(y_pred, 1) == Y).float().mean()
    #print(f"Epoch {epoch}: model accuracy {acc}")

Epoch 0: model accuracy 0.9638333320617676
Epoch 1: model accuracy 0.9590952396392822
Epoch 2: model accuracy 0.9729285836219788
Epoch 3: model accuracy 0.9777619242668152
Epoch 4: model accuracy 0.9831904768943787
Epoch 5: model accuracy 0.9843094944953918
Epoch 6: model accuracy 0.9843094944953918
Epoch 7: model accuracy 0.9886904954910278
Epoch 8: model accuracy 0.9845238327980042
Epoch 9: model accuracy 0.9837142825126648


In [45]:
#Needed to move to testing mode
import torch
import csv
import pandas as pd  # Added pandas import

model.eval()

#Create list to store predictions
ImageId = []
predictions = []

# Convert DataFrame to tensor if X_test is a DataFrame
if isinstance(X_test, pd.DataFrame):
    X_test = torch.tensor(X_test.values, dtype=torch.float32)
# If X_test is a numpy array, convert to tensor
elif isinstance(X_test, np.ndarray):
    X_test = torch.tensor(X_test, dtype=torch.float32)

#Disable gradient calculations
# Create a list to store predictions
with torch.no_grad():
    # Forward pass on test data
    test_outputs = model(X_test)
    _, test_predictions = torch.max(test_outputs.data, 1)
    
    # Convert to numpy arrays
    predictions = test_predictions.cpu().numpy()
    # Create item IDs (adjust as needed for your specific case)
    ImageId = list(range(1,len(predictions)+1))

# Create DataFrame with predictions
results_df = pd.DataFrame({
    'ImageId': ImageId,
    'label': predictions
})

# Save to CSV
results_df.to_csv('ac_multilayer_perceptron predictions1_oct16.csv', index=False)
print("Predictions saved to ac_multilayer_perceptron predictions1_oct16.csv")


Predictions saved to ac_multilayer_perceptron predictions1_oct16.csv
